<h2 align='center'>Codebasics ML Course: ML Flow Tutorial</h2>

In [3]:
%pip install xgboost

import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [5]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

### Experiment 1: Train Logistic Regression Classifier

In [6]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



### Experiment 2: Train Random Forest Classifier

In [7]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       270
           1       0.88      0.70      0.78        30

    accuracy                           0.96       300
   macro avg       0.92      0.84      0.88       300
weighted avg       0.96      0.96      0.96       300



### Experiment 3: Train XGBoost

In [8]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



### Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost

In [9]:
%pip install imblearn
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


(array([0, 1]), array([619, 619]))

In [10]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



<h2 align="center" style="color:blue">Track Experiments Using MLFlow</h2>

In [11]:
models = [
    (
        "Logistic Regression", 
        LogisticRegression(C=1, solver='liblinear'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        RandomForestClassifier(n_estimators=30, max_depth=3), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [12]:
reports = []

for model_name, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [14]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000/")
mlflow.set_experiment("Anamoly Detection")


2024/11/14 17:33:06 INFO mlflow.tracking.fluent: Experiment with name 'Anamoly Detection' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/309657401959101580', creation_time=1731587586149, experiment_id='309657401959101580', last_update_time=1731587586149, lifecycle_stage='active', name='Anamoly Detection', tags={}>

In [15]:


for i , element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]

    with mlflow.start_run(run_name=model_name):
        mlflow.log_param("model_name", model_name)
        mlflow.log_metric("accuracy", report['accuracy'])
        mlflow.log_metric("recall_0", report['0']['recall'])
        mlflow.log_metric("recall_1", report['1']['recall'])
        mlflow.log_metric("macro_f1_score", report['macro avg']['f1-score'])

        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")

2024/11/14 17:33:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/14 17:33:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/309657401959101580/runs/9c6970bc60a6465e9436f2bcda4a0735.
2024/11/14 17:33:15 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/309657401959101580.
2024/11/14 17:33:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/14 17:33:19 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: http://localhost:5000/#/experiments/309657401959101580/runs/24b010361d56423fb6ef2de6326ff766.
2024/11/14 17:33:19 INFO mlflow.tracking._tracking_service.client: 🧪 View exp

<h2 align="center" style="color:blue">Register the Model</h2>

In [16]:
model_name = "XGB-Smote"
model_run_id = input(f"Enter the run id for {model_name}: ")
model_uri = f"runs:/{model_run_id}/model"

result = mlflow.register_model(
    model_uri, model_name
)

Successfully registered model 'XGB-Smote'.
2024/11/14 17:33:52 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.


<h2 align="center" style="color:blue">Load the Model</h2>

In [19]:
model_version = 1
model_uri = f"models:/{model_name}@challenger"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

<h2 align="center" style="color:black">Dev To Production</h2>

In [20]:
dev_model_uri = f"models:/{model_name}@challenger"
prod_model = "Anamoly_Detection_prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=dev_model_uri, dst_name=prod_model)

Successfully registered model 'Anamoly_Detection_prod'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'Anamoly_Detection_prod'.


<ModelVersion: aliases=[], creation_timestamp=1731588079012, current_stage='None', description='', last_updated_timestamp=1731588079012, name='Anamoly_Detection_prod', run_id='998404b237334fffa3d5f2bb37bb4cf7', run_link='', source='models:/XGB-Smote/1', status='READY', status_message='', tags={}, user_id='', version='1'>